In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

set_log_level("ERROR")


In [2]:

target_column = 'Total Inflow hrly'

df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

df

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS
7703,9,108,5,142,14,250,6,39,0,27,...,1.0,1,0,0,3,0,0,2021-07-26 19:00:00,10,18.0
7702,5,113,6,148,11,261,2,41,0,33,...,1.0,1,0,0,3,1,0,2021-07-26 20:00:00,12,9.0
7701,5,118,2,150,7,268,1,42,0,34,...,0.0,1,0,0,4,4,0,2021-07-26 21:00:00,8,4.0
7700,5,123,8,158,13,281,2,44,0,34,...,0.0,1,0,0,3,5,0,2021-07-26 22:00:00,9,6.0
7699,3,126,7,165,10,291,3,47,0,37,...,1.0,0,0,1,3,6,0,2021-07-26 23:00:00,10,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,17,54,8,36,25,90,1,12,1,17,...,0.0,5,2,1,2,13,9,2022-06-14 11:00:00,5,25.0
3,8,62,7,43,15,105,4,16,0,20,...,0.0,4,1,3,4,13,10,2022-06-14 12:00:00,6,29.0
2,13,75,10,53,23,128,1,17,0,20,...,0.0,3,1,1,2,13,10,2022-06-14 13:00:00,2,28.0
1,12,87,8,61,20,148,2,19,0,26,...,0.0,4,0,1,2,12,9,2022-06-14 14:00:00,3,30.0


In [3]:
weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather.ds

0       2020-01-01 00:00:00
1       2020-01-01 01:00:00
2       2020-01-01 02:00:00
3       2020-01-01 03:00:00
4       2020-01-01 04:00:00
                ...        
21501   2022-06-15 19:00:00
21502   2022-06-15 20:00:00
21503   2022-06-15 21:00:00
21504   2022-06-15 22:00:00
21505   2022-06-15 23:00:00
Name: ds, Length: 21506, dtype: datetime64[ns]

In [4]:
df = df.merge(weather, on='ds')


In [5]:
df.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
7686,17,54,8,36,25,90,1,12,1,17,...,9,2022-06-14 11:00:00,5,25.0,22.3,11.9,51.46,0.0,1.8,1013.0
7687,8,62,7,43,15,105,4,16,0,20,...,10,2022-06-14 12:00:00,6,29.0,23.9,12.0,47.03,0.0,3.6,1013.0
7688,13,75,10,53,23,128,1,17,0,20,...,10,2022-06-14 13:00:00,2,28.0,24.0,11.0,44.00,0.0,7.6,1013.0
7689,12,87,8,61,20,148,2,19,0,26,...,9,2022-06-14 14:00:00,3,30.0,23.9,12.0,47.00,0.0,5.4,1012.7
7690,8,95,13,74,21,169,3,22,0,30,...,8,2022-06-14 15:00:00,3,36.0,26.5,11.0,37.91,0.0,12.2,1013.0


In [6]:
weather_regressors = ['temp','dew','humidity','precip','windspeed','sealevelpressure']

In [7]:

m = NeuralProphet(
  # growth='off',
  yearly_seasonality=False,
  weekly_seasonality=True,
  daily_seasonality=True,
  n_lags=48,
  n_forecasts=12,
  changepoints_range=0.95,
  n_changepoints=50,
  num_hidden_layers=4,
  d_hidden=36,
  learning_rate=0.005,
)
for reg in weather_regressors:
  m = m.add_future_regressor(name=reg)
m = m.add_lagged_regressor(names=regressors)
m = m.add_country_holidays("CA")
metrics = m.fit(df, 
                freq='H', 
                # progress='plot'
                )
print(metrics.tail(1))

with open('../models/inflow_forecast_model_with_weather.pkl', "wb") as f:
    pickle.dump(m, f)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neur

     SmoothL1Loss       MAE      RMSE  RegLoss
109      0.002482  1.133517  1.476477      0.0


In [14]:
loaded_model = pickle.load(open('../models/inflow_forecast_model_with_weather.pkl', 'rb'))


In [15]:
last_timestamp = df.loc[len(df)-1].ds
last_timestamp

Timestamp('2022-06-14 15:00:00')

In [16]:
weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather[weather.ds>last_timestamp].head(12)

,ds,temp,dew,humidity,precip,windspeed,sealevelpressure
21474,2022-06-14 16:00:00,26.7,10.7,36.72,0.0,13.0,1013.0
21475,2022-06-14 17:00:00,26.6,11.2,38.19,0.0,12.2,1013.0
21476,2022-06-14 18:00:00,26.4,11.4,39.16,0.0,13.0,1014.0
21477,2022-06-14 19:00:00,25.6,12.3,43.57,0.0,13.0,1014.0
21478,2022-06-14 20:00:00,24.0,13.1,50.52,0.0,9.7,1014.0
21479,2022-06-14 21:00:00,21.9,13.7,59.66,0.0,6.8,1015.0
21480,2022-06-14 22:00:00,20.7,13.9,65.05,0.0,5.8,1015.0
21481,2022-06-14 23:00:00,20.1,14.0,67.95,0.0,7.2,1015.0
21482,2022-06-15 00:00:00,19.2,14.3,73.26,0.0,5.8,1016.0
21483,2022-06-15 01:00:00,18.2,15.0,81.59,0.0,7.2,1017.0


In [17]:
future = pd.concat([df.tail(48),weather[weather.ds>last_timestamp].head(12)])
future.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
21481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-14 23:00:00,NaN,NaN,20.1,14.0,67.95,0.0,7.2,1015.0
21482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-15 00:00:00,NaN,NaN,19.2,14.3,73.26,0.0,5.8,1016.0
21483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-15 01:00:00,NaN,NaN,18.2,15.0,81.59,0.0,7.2,1017.0
21484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-15 02:00:00,NaN,NaN,17.1,14.6,85.23,0.0,7.9,1017.0
21485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-15 03:00:00,NaN,NaN,16.4,14.0,85.71,0.0,7.6,1018.0


In [18]:
forecast = loaded_model.predict(future, raw=True, decompose=False)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:1649: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



In [19]:




start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='H')
forecast_output['inflow'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('../forecasts/inflow_with_weather.csv', index=False)
forecast_output

,ds,inflow,timestamp
0,2022-06-14 12:00:00,20.008987,2022-06-14 19:55:21
1,2022-06-14 13:00:00,24.327162,2022-06-14 19:55:21
2,2022-06-14 14:00:00,22.113440,2022-06-14 19:55:21
3,2022-06-14 15:00:00,22.790991,2022-06-14 19:55:21
4,2022-06-14 16:00:00,20.641260,2022-06-14 19:55:21
5,2022-06-14 17:00:00,17.598457,2022-06-14 19:55:21
6,2022-06-14 18:00:00,14.596188,2022-06-14 19:55:21
7,2022-06-14 19:00:00,17.440479,2022-06-14 19:55:21
8,2022-06-14 20:00:00,15.678611,2022-06-14 19:55:21
9,2022-06-14 21:00:00,14.005450,2022-06-14 19:55:21
